In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np

import torchviz

print(torch.cuda.is_available)


<function is_available at 0x000001F5448FF420>


In [11]:
class RegressionDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data
        self.n_samples = x_data.shape[0]
        
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

In [14]:
x_numpy = np.array([-1.5, -0.8, 0.1, 0.9, 1.7], dtype=np.float32)
y_numpy = np.array([0.3, -0.3, 0.5, 1.8, 1.5], dtype=np.float32)

x_input_tensor = torch.from_numpy(x_numpy).unsqueeze(1) # Add a dimension for features (N, 1)
y_target_tensor = torch.from_numpy(y_numpy).unsqueeze(1) # Add a dimension for features (N, 1)

print(x_input_tensor)
print(x_input_tensor.shape)
print(y_target_tensor)
print(y_target_tensor.shape)


dataset = RegressionDataset(x_input_tensor, y_target_tensor)
batch_size = len(dataset) # Use full batch size for the dataset
print(batch_size)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False) # shuffle=False as order doesn't matter for full batch

tensor([[-1.5000],
        [-0.8000],
        [ 0.1000],
        [ 0.9000],
        [ 1.7000]])
torch.Size([5, 1])
tensor([[ 0.3000],
        [-0.3000],
        [ 0.5000],
        [ 1.8000],
        [ 1.5000]])
torch.Size([5, 1])
5


In [20]:
class SimpleModelWParam(nn.Module):
    def __init__(self, init_weight=-10.0, init_bias=10.0):
        super().__init__()
        
        # Trainable Param
        self.weight = nn.Parameter(torch.tensor([init_weight], dtype=torch.float32))
        self.bias = nn.Parameter(torch.tensor([init_bias], dtype=torch.float32))
        
    def forward(self, x):
        return x * self.weight + self.bias
    
class SimpleModelWModule(nn.Module):
    def __init__(self, init_weight=-10.0, init_bias=10.0):
        super().__init__()
        
        self.linear = nn.Linear(in_features=1, out_features=1, bias=True)
        
        with torch.no_grad(): # Disable gradient calculation during init
            self.linear.weight.fill_(init_weight)
            self.linear.bias.fill_(init_bias)
            
    def forward(self, x):
        return self.linear(x)
    
model = SimpleModelWModule()

In [21]:
criterion = nn.MSELoss() # MSE (Mean Square Error)

optimizer = optim.SGD(model.parameters(), lr=0.1)
num_epochs = 50

losses = [] 
for epoch in range(num_epochs):
    for x_inputs, y_targets in dataloader:
        
        y_pred = model(x_inputs)
        loss = criterion(y_pred, y_targets)
        losses.append(loss.item())
        
        # perform a backward pass, and upate the parameter
        optimizer.zero_grad() # clear previous gradient of model.parameters(), zeroing gradient
        loss.backward()
        
        optimizer.step()
        
        if epoch == 0:
             # Visualizes the graph starting from the 'loss' tensor.
             # 'params' helps in labeling the nodes with parameter names.
             graph = torchviz.make_dot(loss, params=dict(model.named_parameters()))
             graph.render("autograd_graph", view=False) # Renders and optionally opens the graph file
            
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}: "
              f"w = {model.linear.weight.item():.4f}, "
              f"b = {model.linear.bias.item():.4f}, "
              f"Loss = {loss.item():.4f}"
)
        

Epoch 10: w = -0.0835, b = 1.8877, Loss = 3.0419
Epoch 20: w = 0.5153, b = 0.8525, Loss = 0.2208
Epoch 30: w = 0.5598, b = 0.7306, Loss = 0.1886
Epoch 40: w = 0.5638, b = 0.7166, Loss = 0.1882
Epoch 50: w = 0.5642, b = 0.7151, Loss = 0.1882
